## **AI Health Checker**

In [6]:

import re
import difflib
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ---------------------------
# Disease database (your logic)
# ---------------------------
DATABASE = {
    "Common Cold": {
        "keywords": ["cough", "sneeze", "fever", "congestion", "runny nose", "sore throat", "throat hurts", "body ache"],
        "advice": "Rest, drink fluids, and take OTC cold medicine if needed."
    },
    "Flu": {
        "keywords": ["fever", "chills", "fatigue", "headache", "muscle aches", "body pain", "back pain", "tiredness"],
        "advice": "Take care of yourself, stay hydrated, and relax."
    },
    "Migraine": {
        "keywords": ["headache", "nausea", "sensitivity to light", "sensitivity to sound", "throbbing headache", "migraine pain"],
        "advice": "Rest in a dark room, take your meds, and stay calm."
    },
    "COVID-19": {
        "keywords": ["fever", "cough", "tiredness", "loss of taste", "loss of smell", "shortness of breath", "difficulty breathing"],
        "advice": "Get tested, stay safe, and isolate if needed."
    },
    "Asthma": {
        "keywords": ["wheezing", "shortness of breath", "difficulty breathing", "chest tightness"],
        "advice": "Consult a doctor for inhalers or asthma management plan."
    },
    "Heart Attack": {
        "keywords": ["chest pain", "pressure on heart", "tightness in chest", "arm pain", "jaw pain", "shortness of breath", "suffocation"],
        "advice": "Call emergency services immediately!"
    },
    "Anemia": {
        "keywords": ["weakness", "fatigue", "laziness", "paleness", "shortness of breath"],
        "advice": "Consult a doctor for blood tests and iron supplements."
    },
    "Stomach Infection": {
        "keywords": ["nausea", "vomiting", "diarrhea", "cramps", "abdominal pain", "fever"],
        "advice": "Drink fluids, eat light, and rest well."
    }
}

# ---------------------------
# Symptom matching functions (your logic)
# ---------------------------
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    return text

def match_symptom(input_text, keyword):
    keyword = keyword.lower().strip()
    input_text = input_text.lower()
    if keyword in input_text:
        return True, len(keyword)
    words = input_text.split()
    closest = difflib.get_close_matches(keyword, words, n=1, cutoff=0.6)
    if closest:
        return True, len(keyword)
    return False, 0

def predict_disease(input_text, database):
    results = {}
    urgent_symptoms = ["pressure on heart", "chest pain", "difficulty breathing", "shortness of breath", "suffocation"]
    urgent_matches = []

    for disease, info in database.items():
        score_sum = 0
        keywords = info.get("keywords", [])
        for keyword in keywords:
            matched, weight = match_symptom(input_text, keyword)
            if matched:
                score_sum += weight
                if keyword in urgent_symptoms:
                    urgent_matches.append((disease, keyword))
        if keywords:
            max_score = sum(len(k) for k in keywords)
            score = int((score_sum / max_score) * 100)
            if score > 0:
                results[disease] = {"score": score, "advice": info["advice"]}
    
    # Sort results
    sorted_results = dict(sorted(results.items(), key=lambda x: x[1]["score"], reverse=True))
    return urgent_matches, sorted_results

# ---------------------------
# Layout & design (from your widget code)
# ---------------------------
def display_results(results, urgent=[]):
    colors = ["#ff6f61","#ffb347","#6a82fb","#56ab2f","#ff7eb3"]
    html = ""
    
    # Urgent symptoms at the top
    if urgent:
        html += "<div style='background:#ff4c4c; border-radius:20px; padding:20px; color:white; margin-bottom:15px;'>"
        html += "<div style='font-size:18px; font-weight:bold;'>⚠️ URGENT SYMPTOMS DETECTED! SEEK MEDICAL ATTENTION IMMEDIATELY! ⚠️</div>"
        for disease, symptom in urgent:
            html += f"<div>- {disease}: {symptom}</div>"
        html += "</div>"
    
    i = 0
    for disease, info in results.items():
        color = colors[i % len(colors)]
        html += f"""
        <div style='background:{color}; border-radius:20px; padding:20px; color:white; margin-bottom:15px;'>
            <div style='font-size:20px; font-weight:bold;'>{disease} ({info['score']}% match)</div>
            <div style='margin:10px 0; font-size:16px;'>Recommendation: {info['advice']}</div>
            <div style='width:100%; background: rgba(255,255,255,0.3); border-radius:10px;'>
                <div style='width:{info['score']}%; background: rgba(255,255,255,0.8); height:15px; border-radius:10px;'></div>
            </div>
        </div>
        """
        i += 1
    display(HTML(html))

# ---------------------------
# Widgets
# ---------------------------
symptom_input = widgets.Textarea(
    placeholder='Describe your symptoms here...',
    layout=widgets.Layout(width='90%', height='80px')
)
check_button = widgets.Button(
    description='Check Now!',
    layout=widgets.Layout(width='200px', height='50px'),
    style={'button_color':'#ff758c', 'font_weight':'bold'}
)
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        user_symptoms = symptom_input.value
        urgent, predictions = predict_disease(clean_text(user_symptoms), DATABASE)
        display_results(predictions, urgent)

check_button.on_click(on_button_click)

display(symptom_input, check_button, output)


Textarea(value='', layout=Layout(height='80px', width='90%'), placeholder='Describe your symptoms here...')

Button(description='Check Now!', layout=Layout(height='50px', width='200px'), style=ButtonStyle(button_color='…

Output()